In [8]:
import pandas as pd
from resources.data import get_dataset
from resources.execution import create_standard_fully_connected_nn
from resources.rules import get_rules, FEATURE_MAPPING as POKER_FEATURE_MAPPING, CLASS_MAPPING as POKER_CLASS_MAPPING
from psyki.ski import Injector, Formula
from psyki.logic.datalog.grammar.adapters import antlr4
from latency import LatencyQoS


## From the experiment of KILL 
https://github.com/MatteoMagnini/kill-experiments-woa-2022

In [9]:
data = pd.DataFrame(get_dataset('train'), dtype = 'int32')

In [10]:
model = create_standard_fully_connected_nn(10, 10, 3, 128, 'relu')

2022-10-10 18:40:24.421385: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-10-10 18:40:25.168204: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1532] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 6668 MB memory:  -> device: 0, name: Quadro RTX 4000, pci bus id: 0000:17:00.0, compute capability: 7.5
2022-10-10 18:40:25.169212: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1532] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 6594 MB memory:  -> device: 1, name: Quadro RTX 4000, pci bus id: 0000:65:00.0, compute capability: 7.5


In [11]:
injector = Injector.kill(model, POKER_CLASS_MAPPING, POKER_FEATURE_MAPPING)


In [12]:
formulae = [antlr4.get_formula_from_string(rule) for rule in get_rules()]

In [13]:
options = {'optim':'adam',
           'loss': 'sparse_categorical_crossentropy',
           'batch': 32,
           'epochs':2,
           'dataset': data,
           'formula': formulae}

In [14]:
latency = LatencyQoS(model, injector, options)

In [15]:
latency.measure(fit = True)

Measuring times of model training. This can take a while as model.fit needs to run...
The injected model is 61.29815 seconds slower during training
Measuring times of model prediction. This may take a while depending on the model and dataset...
The injected model is 2.78592 seconds slower during inference
